In [5]:
import requests
from bs4 import BeautifulSoup
import json

# Define the URL of the page to scrape
url = "https://pih.com.pk/doctors"

try:
    response = requests.get(url)
    response.raise_for_status()  # Check for HTTP errors
    html = response.text
except requests.RequestException as e:
    print(f"Error fetching the page: {e}")
    exit()

soup = BeautifulSoup(html, 'html.parser')

doctors = []
seen_doctors = set()  # To track and remove duplicates

for element in soup.find_all(class_='elementor-widget-wrap'):
    # Extract name
    name_tag = element.find('h3')
    if name_tag:
        name = name_tag.text.strip()

        # Skip if the doctor has already been added
        if name in seen_doctors:
            continue
        seen_doctors.add(name)

        # Extract education and position
        text_editor = element.find(class_='elementor-widget-text-editor')
        if text_editor:
            lines = text_editor.text.strip().split('\n')
            education = lines[0].strip()
            position = lines[1].strip() if len(lines) > 1 else "N/A"

            # Extract days and timings
            clinic_info = lines[2:] if len(lines) > 2 else []
            days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
            timing = {day: clinic_info[0] if clinic_info else "Not Available" for day in days}

            doctor = {
                "name": name,
                "education": education,
                "hospital": "Pak International Hospital",  # Assuming this is static; update if dynamic
                "position": position,
                "clinic": {
                    "days": days,
                    "timing": timing
                }
            }

            doctors.append(doctor)

print(json.dumps(doctors, indent=2))


[
  {
    "name": "Prof. Dr. M. Iqbal Afridi",
    "education": "MBBS, MCPS PSYCH, FCPS PSYCH (PAK),FACP (USA), FRCP (Ireland),Consultant Neuro-psychiatrist & AddictionologistMonday to Saturday 6:00 Pm to 10:00 Pm",
    "hospital": "Pak International Hospital",
    "position": "N/A",
    "clinic": {
      "days": [
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday"
      ],
      "timing": {
        "Monday": "Not Available",
        "Tuesday": "Not Available",
        "Wednesday": "Not Available",
        "Thursday": "Not Available",
        "Friday": "Not Available",
        "Saturday": "Not Available"
      }
    }
  },
  {
    "name": "Rozeena Dharwarwala",
    "education": "Clinical Psychologist & HypnotherapistM.Phil (Clinical Psychology) ICD, KU.Forensic Mental Health Training SOFMH, NHS Scotland.Monday to Saturday \u2013 4 Pm to 10 Pm",
    "hospital": "Pak International Hospital",
    "position": "N/A",
    "clini